Title: Analyzing Sensor Data with Spark Steaming

Author: Ivan Zheng

Date: 10/01/2017

In [1]:
# Example weather station data
#
# 1419408015	0R1,Dn=059D,Dm=066D,Dx=080D,Sn=8.5M,Sm=9.5M,Sx=10.3M
# 1419408016	0R1,Dn=059D,Dm=065D,Dx=078D,Sn=8.5M,Sm=9.5M,Sx=10.3M
# 1419408016	0R2,Ta=13.9C,Ua=28.5P,Pa=889.9H
# 1419408017	0R1,Dn=059D,Dm=064D,Dx=075D,Sn=8.7M,Sm=9.6M,Sx=10.3M
# 1419408018	0R1,Dn=059D,Dm=064D,Dx=075D,Sn=8.9M,Sm=9.6M,Sx=10.3M
# 1419408019	0R1,Dn=059D,Dm=065D,Dx=075D,Sn=8.8M,Sm=9.5M,Sx=10.3M

Each line contains a timestamp and a set of measurements. Each measurement has an abbreviation, and for this exercise, we are interested in the average wind direction, which is Dm. The next cell lists the abbreviations used for each type of measurement:

In [2]:
# Key for measurements:
#
# Sn      Wind speed minimum m/s, km/h, mph, knots #,M, K, S, N
# Sm      Wind speed average m/s, km/h, mph, knots #,M, K, S, N
# Sx      Wind speed maximum m/s, km/h, mph, knots #,M, K, S, N
# Dn      Wind direction minimum deg #, D
# Dm      Wind direction average deg #, D
# Dx      Wind direction maximum deg #, D
# Pa      Air pressure hPa, Pa, bar, mmHg, inHg #, H, P, B, M, I
# Ta      Air temperature °C, °F #, C, F
# Tp      Internal temperature °C, °F #, C, F
# Ua      Relative humidity %RH #, P
# Rc      Rain accumulation mm, in #, M, I
# Rd      Rain duration s #, S
# Ri      Rain intensity mm/h, in/h #, M, I
# Rp      Rain peak intensity mm/h, in/h #, M, I
# Hc      Hail accumulation hits/cm2, hits/in2, hits #, M, I, H
# Hd      Hail duration s #, S
# Hi      Hail intensity hits/cm2h, hits/in2h, hits/ h #, M, I, H
# Hp      Hail peak intensity hits/cm2h, hits/in2h, hits/ h #, M, I, H
# Th      Heating temperature °C, °F #, C, F
# Vh      Heating voltage V #, N, V, W, F2
# Vs      Supply voltage V V
# Vr      3.5 V ref. voltage V V

The third cell defines a function that parses each line and returns the average wind direction (Dm). Run this cell:

In [3]:
# Parse a line of weather station data, returning the average wind direction measurement 
#
import re
def parse(line):
    match = re.search("Dm=(\d+)", line)
    if match:
        val = match.group(1)
        return [int(val)]
    return []

Step 1. Import and create streaming context. Next, we will import and create a new instance of Spark's StreamingContext:

In [4]:
from pyspark.streaming import StreamingContext
ssc = StreamingContext(sc,1)

Similar to the SparkContext, the StreamingContext provides an interface to Spark's streaming capabilities. The argument sc is the SparkContext, and 1 specifies a batch interval of one second.

Step 2. Create DStream of weather data. Let's open a connection to the streaming weather data:

In [5]:
lines = ssc.socketTextStream("rtd.hpwren.ucsd.edu", 12028)

This create a new variable, lines, to be a Spark DStream that streams the lines of output from the weather station

Step 3. Read measurement. Next, let's read the average wind speed from each line and store it in a new DStream vals:

In [6]:
vals = lines.flatMap(parse)

This line uses flatMap() to iterate over the lines DStream, and calls the parse() function we defined above to get the average wind speed.

Step 4. Create sliding window of data. We can create a sliding window over the measurements by calling the window() method:

In [7]:
window = vals.window(10,5)

This create a new DStream called window that combines the ten seconds worth of data and moves by five seconds.

Step 5. Define and call analysis function. We would like to find the minimum and maximum values in our window. Let's define a function that prints these values for an RDD:

In [8]:
def stats(rdd):
    print(rdd.collect())
    if rdd.count() > 0:
        print("max = {}, min = {}".format(rd.max(), rdd.min()))

This function first prints the entire contents of the RDD by calling the collect() method. This is done to demonstrate the sliding window and would not be practical if the RDD was containing a large amount of data. Next, we check if the size of the RDD is greater than zero before printing the maximum and minimum values.

 Next, we call the stats() function for each RDD in our sliding window:

In [9]:
window.foreachRDD(lambda rdd: stats(rdd))

 This line calls the stats() function defined above for each RDD in the DStream window.

Step 6. Start the stream processing. We call start() on the StreamingContext to begin the processing:

In [10]:
ssc.start()

[]
[]
[]
[]
[]
[]


The sliding window contains ten seconds worth of data and slides every five seconds. In the beginning, the number of values in the windows are increasing as the data accumulates, and after Window 3, the size stays (approximately) the same. Since the window slides half as often as the size of the window, the second half of a window becomes the first half of the next window. For example, the second half of Window 5 is 310, 321, 323, 325, 326, which becomes the first half of Window 6.

When we are done, call stop() on the StreamingContext:

In [11]:
ssc.stop()